In [ ]:
TF_VERSION = "2.1.0"
DEBUG = True
COLAB = True
BASE_DIR = os.getcwd()
COLAB_DIR = "/content/drive/MyDrive/"

if COLAB:
  BASE_DIR = COLAB_DIR
  from google.colab import drive
  drive.mount("/content/drive")

In [1]:
from abc import ABCMeta, abstractmethod
import numpy as np
import matplotlib.pyplot as plt
import cv2, os, itertools, random, pickle, warnings, datetime, math, gc, copy
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, applications, callbacks, metrics
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU, PReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import L1L2, l2, l1, l1_l2
from tensorflow.keras.models import load_model

In [7]:
def LoadWorkState(statePickle):
  f = open(statePickle, "rb")
  L = pickle.load(f)
  f.close()
  return L


def StoreWorkState(L, statePickle):
  f = open(statePickle, "wb")
  L = pickle.dump(L, f)
  f.close()


def GetCurrentState():
  L = {
    "TF_VERSION": TF_VERSION,
    "DEBUG": DEBUG,
    "COLAB": COLAB,
    "BASE_DIR": BASE_DIR,
    "COLAB_DIR": COLAB_DIR,
    "OWNER": OWNER,
    "PBO": PBO,
    "MODEL": MODEL,
    "DATASET_PICKLE_FILE": DATASET_PICKLE_FILE,
    "PICKELS_BASE_DIR": PICKELS_BASE_DIR,
    "RESULTS_BASE_DIR": RESULTS_BASE_DIR,
    "IMAGE_SHAPE": IMAGE_SHAPE,
    "CATEGORIES": CATEGORIES,
    "ITERS": ITERS,
    "EPOCHS": EPOCHS,
    "POPULATION_SIZE": POPULATION_SIZE,
    "DIM": DIM,
    "SKIP": SKIP,
    "POPULATION": POPULATION,
    "IS_NEW": IS_NEW,
    "TRAIN_RATIO": TRAIN_RATIO,
    "DROPOUT_RANGE": DROPOUT_RANGE,
    "OUTPUT_ACTIVATION": OUTPUT_ACTIVATION,
    "PRETRAINED_WEIGHTS": PRETRAINED_WEIGHTS,
    "BATCH_SIZES": BATCH_SIZES,
    "LEARN_RATIOS": LEARN_RATIOS,
    "OPTIMIZERS": OPTIMIZERS,
    "HIDDEN_ACTIVATIONS": HIDDEN_ACTIVATIONS,
    "WEIGHT_INITIALIZERS": WEIGHT_INITIALIZERS,
    "RESULTS_DIR": RESULTS_DIR,
    "DATASET_PICKLE": DATASET_PICKLE,
    "OWNER_DIR": OWNER_DIR,
    "HDF5_RESULTS_DIR": HDF5_RESULTS_DIR,
    "FIGURES_RESULTS_DIR": FIGURES_RESULTS_DIR,
    "HISTORY_RESULTS_DIR": HISTORY_RESULTS_DIR,
    "STATE_PICKLE": STATE_PICKLE,
    "LOGGER_FILE": LOGGER_FILE,
    "BEST_FILE": BEST_FILE,
    "SPLIT_DATASET_PICKLE": SPLIT_DATASET_PICKLE,
    "AUGMENTATION_CONFIG": AUGMENTATION_CONFIG,
    "GA": {},
    "INTERPOLATION": INTERPOLATION,
    "USE_TF": USE_TF,
    "REGULARIZERS": REGULARIZERS,
  }
  return L


def LoadCreateSplitFile(datasetPickle,
            splitPickle,
            noOfClasses,
            trainRatio,
            debug=True):
  if (os.path.exists(splitPickle)):
    f = open(splitPickle, "rb")
    [Xtrain, Xvalidation, Xtest, Ytrain, Yvalidation,
     Ytest] = pickle.load(f)
    f.close()
    Q = [Xtrain, Xvalidation, Xtest, Ytrain, Yvalidation, Ytest]
  else:
    randomState = 42
    f = open(datasetPickle, "rb")
    dataset = pickle.load(f)
    f.close()
    random.shuffle(dataset)
    images, classes = zip(*dataset)
    images = np.array(images, dtype='float32')
    images = (images / 255.0).astype('float32')
    classes = to_categorical(classes, noOfClasses)
    classes = np.array(classes)
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(
      images, classes, train_size=trainRatio, random_state=randomState)
    Xtrain, Xvalidation, Ytrain, Yvalidation = train_test_split(
      Xtrain, Ytrain, train_size=trainRatio, random_state=randomState)
    Q = [Xtrain, Xvalidation, Xtest, Ytrain, Yvalidation, Ytest]
    f = open(splitPickle, "wb")
    pickle.dump(Q, f)
    f.close()
  return Q


def HandleCurrentState(isNew, statePickle, loggerFile):
  if (isNew):
    currentState = GetCurrentState()
    StoreWorkState(currentState, statePickle)
  else:
    if (os.path.exists(statePickle)):
      currentState = LoadWorkState(statePickle)
    else:
      currentState = GetCurrentState()
      StoreWorkState(currentState, statePickle)
  return currentState


def recall_m(yTrue, yPred):
  truePositives = K.sum(K.round(K.clip(yTrue * yPred, 0, 1)))
  possiblePositives = K.sum(K.round(K.clip(yTrue, 0, 1)))
  recall = truePositives / (possiblePositives + K.epsilon())
  return recall


def precision_m(yTrue, yPred):
  truePositives = K.sum(K.round(K.clip(yTrue * yPred, 0, 1)))
  predictedPositives = K.sum(K.round(K.clip(yPred, 0, 1)))
  precision = truePositives / (predictedPositives + K.epsilon())
  return precision


def f1_m(yTrue, yPred):
  precision = precision_m(yTrue, yPred)
  recall = recall_m(yTrue, yPred)
  return 2.0 * ((precision * recall) / (precision + recall + K.epsilon()))


def BuildCompileTransferLearningCNN(model,
                  inputShape,
                  noOfClasses,
                  hiddenActivation,
                  outputActivation,
                  weightInitializer,
                  pretrainedWeights,
                  optimizer,
                  dropout,
                  learnRatio,
                  regularizer,
                  debug=True):

  baseModel = None
  includeTop = False

  if (model == "MobileNetV2"):
    baseModel = applications.MobileNetV2(weights=pretrainedWeights,
                       include_top=includeTop)
  elif (model == "MobileNet"):
    baseModel = applications.MobileNet(weights=pretrainedWeights,
                       include_top=includeTop)
  elif (model == "VGG16"):
    baseModel = applications.VGG16(weights=pretrainedWeights,
                     include_top=includeTop)
  elif (model == "VGG19"):
    baseModel = applications.VGG19(weights=pretrainedWeights,
                     include_top=includeTop)
  elif (model == "ResNet50"):
    baseModel = applications.ResNet50(weights=pretrainedWeights,
                      include_top=includeTop)
  elif (model == "ResNet101"):
    baseModel = applications.ResNet101(weights=pretrainedWeights,
                       include_top=includeTop)
  elif (model == "Xception"):
    baseModel = applications.Xception(weights=pretrainedWeights,
                      include_top=includeTop)
  elif (model == "DenseNet121"):
    baseModel = applications.DenseNet121(weights=pretrainedWeights,
                       include_top=includeTop)
  elif (model == "DenseNet169"):
    baseModel = applications.DenseNet169(weights=pretrainedWeights,
                       include_top=includeTop)

  if (learnRatio is None): learnRatio = 100.0

  for layer in baseModel.layers:
    layer.trainable = False
  L = len(baseModel.layers)
  fromL = int(learnRatio * L / 100.0)
  for layer in baseModel.layers[L - fromL:]:
    layer.trainable = True

  if (dropout is None or dropout <= 0):
    model = tf.keras.Sequential([
      baseModel,
      layers.GlobalAveragePooling2D(),
      layers.Dense(noOfClasses, activation=outputActivation)
    ])
  else:
    model = tf.keras.Sequential([
      baseModel,
      layers.GlobalAveragePooling2D(),
      layers.Dropout(dropout),
      layers.Dense(noOfClasses, activation=outputActivation)
    ])

  metricsList = [
    'accuracy', f1_m, precision_m, recall_m,
    metrics.AUC(),
    metrics.Precision(),
    metrics.Recall()
  ]
  loss = "categorical_crossentropy"
  if (optimizer is not None):
    model.compile(optimizer=optimizer, loss=loss, metrics=metricsList)
  else:
    model.compile(loss=loss, metrics=metricsList)
  return model


def BuildCompileNativeCNN(model,
              inputShape,
              noOfClasses,
              hiddenActivation,
              outputActivation,
              weightInitializer,
              pretrainedWeights,
              optimizer,
              dropout,
              learnRatio,
              regularizer,
              debug=True):

  baseModel = None

  def __generateConv2D(noOfFilters,
             kernelSize,
             strideSize,
             hiddenActivation,
             weightInitializer,
             regularizer,
             inputShape=None):
    if (weightInitializer is None): weightInitializer = "glorot_uniform"
    if (inputShape is None):
      return layers.Conv2D(noOfFilters,
                 kernel_size=kernelSize,
                 strides=strideSize,
                 padding='same',
                 activation=hiddenActivation,
                 kernel_initializer=weightInitializer,
                 kernel_regularizer=regularizer)
    else:
      return layers.Conv2D(noOfFilters,
                 kernel_size=kernelSize,
                 strides=strideSize,
                 padding='same',
                 activation=hiddenActivation,
                 kernel_initializer=weightInitializer,
                 kernel_regularizer=regularizer,
                 input_shape=inputShape)

  if (model == "HMB1-COVID19"):
    baseModel = tf.keras.Sequential()
    for i, k in enumerate([32, 64, 128]):
      if (i == 0):
        baseModel.add(
          __generateConv2D(k, (3, 3), (1, 1), hiddenActivation,
                   weightInitializer, regularizer,
                   inputShape))
      else:
        baseModel.add(
          __generateConv2D(k, (3, 3), (1, 1), hiddenActivation,
                   weightInitializer, regularizer, None))
      baseModel.add(layers.BatchNormalization())
      baseModel.add(
        __generateConv2D(k, (3, 3), (1, 1), hiddenActivation,
                 weightInitializer, regularizer))
      baseModel.add(layers.BatchNormalization())
      baseModel.add(layers.MaxPooling2D(pool_size=2))
      if (dropout is not None and dropout > 0):
        baseModel.add(layers.Dropout(dropout))

    baseModel.add(layers.Flatten())

  if (learnRatio is None): learnRatio = 100.0

  for layer in baseModel.layers:
    layer.trainable = False
  L = len(baseModel.layers)
  fromL = int(learnRatio * L / 100.0)
  for layer in baseModel.layers[L - fromL:]:
    layer.trainable = True

  model = tf.keras.Sequential(
    [baseModel,
     layers.Dense(noOfClasses, activation=outputActivation)])

  metricsList = [
    'accuracy', f1_m, precision_m, recall_m,
    metrics.AUC(),
    metrics.Precision(),
    metrics.Recall()
  ]
  loss = "categorical_crossentropy"
  if (optimizer is not None):
    model.compile(optimizer=optimizer, loss=loss, metrics=metricsList)
  else:
    model.compile(loss=loss, metrics=metricsList)
  return model


def MapSolutionAsString(currentState, solution):
  mappedSolution = None
  if (isinstance(currentState["DIM"], int)):
    mappedSolution = [None, None, None, None, None, None, None]
    for i in range(currentState["DIM"]):
      mappedSolution[i] = "auto"
  elif (isinstance(currentState["DIM"], list)
      or isinstance(currentState["DIM"], tuple)):
    mappedSolution = copy.copy(currentState["DIM"])
    for i in range(len(mappedSolution)):
      if (mappedSolution[i] == "N/A"): mappedSolution[i] = None
  if (mappedSolution[0] == "auto"):
    r1 = math.floor(solution[0] * (len(currentState["OPTIMIZERS"]) - 1))
    optimizer = currentState["OPTIMIZERS"][r1]
    mappedSolution[0] = optimizer
  if (mappedSolution[1] == "auto"):
    r2 = math.floor(solution[1] * (len(currentState["BATCH_SIZES"]) - 1))
    batchSize = currentState["BATCH_SIZES"][r2]
    mappedSolution[1] = batchSize
  if (mappedSolution[2] == "auto"):
    low = currentState["DROPOUT_RANGE"][0]
    high = currentState["DROPOUT_RANGE"][1]
    delta = high - low
    dropout = round(solution[2] * delta * 10.0) / 100.0
    mappedSolution[2] = dropout
  if (mappedSolution[3] == "auto"):
    r3 = math.floor(solution[3] * (len(currentState["LEARN_RATIOS"]) - 1))
    learnRatio = currentState["LEARN_RATIOS"][r3]
    mappedSolution[3] = learnRatio
  if (mappedSolution[4] == "auto"):
    r4 = math.floor(solution[4] *
            (len(currentState["HIDDEN_ACTIVATIONS"]) - 1))
    activation = currentState["HIDDEN_ACTIVATIONS"][r4]
    mappedSolution[4] = activation
  if (mappedSolution[5] == "auto"):
    r5 = math.floor(solution[5] *
            (len(currentState["WEIGHT_INITIALIZERS"]) - 1))
    weightInitializer = currentState["WEIGHT_INITIALIZERS"][r5]
    mappedSolution[5] = weightInitializer
  if (mappedSolution[6] == "auto"):
    r6 = math.floor(solution[6] * (len(currentState["REGULARIZERS"]) - 1))
    regularizer = currentState["REGULARIZERS"][r6]
    mappedSolution[6] = regularizer
  return copy.copy(mappedSolution)


def MapSolutionForCNN(currentState, solution):
  mappedSolution = MapSolutionAsString(currentState, solution)

  if (mappedSolution[4] == "prelu"):
    activation = PReLU()
  elif (mappedSolution[4] == "leakyrelu"):
    activation = LeakyReLU()

  if (mappedSolution[6] is not None):
    splitted = mappedSolution[6].split("_")
    if (splitted[0] == "l1"): regularizer = l1(float(splitted[1]))
    elif (splitted[0] == "l2"): regularizer = l2(float(splitted[1]))
    else: regularizer = None
    mappedSolution[6] = regularizer

  return copy.copy(mappedSolution)

In [4]:
def FitnessFunction(iteration, solution, currentState):
  K.clear_session()
  np.random.seed(42)
  tf.random.set_seed(42)
  gc.collect()

  mappedSolution = MapSolutionForCNN(currentState, solution)
  optimizer, batchSize, dropout, learnRatio, hiddenActivation, weightInitializer, regularizer = mappedSolution

  aug = ImageDataGenerator(
    rotation_range=currentState["AUGMENTATION_CONFIG"]['rotation_range'],
    width_shift_range=currentState["AUGMENTATION_CONFIG"]
    ['width_shift_range'],
    height_shift_range=currentState["AUGMENTATION_CONFIG"]
    ['height_shift_range'],
    shear_range=currentState["AUGMENTATION_CONFIG"]['shear_range'],
    zoom_range=currentState["AUGMENTATION_CONFIG"]['zoom_range'],
    horizontal_flip=currentState["AUGMENTATION_CONFIG"]['horizontal_flip'],
    vertical_flip=currentState["AUGMENTATION_CONFIG"]['vertical_flip'],
    fill_mode=currentState["AUGMENTATION_CONFIG"]['fill_mode'],
  )

  trainIter = aug.flow(Xtrain, Ytrain, batch_size=batchSize)
  owner = currentState["OWNER"]
  modelName = currentState["MODEL"]
  inputShape = currentState["IMAGE_SHAPE"]
  noOfClasses = len(currentState["CATEGORIES"])

  if (isinstance(optimizer, LeakyReLU) or isinstance(optimizer, PReLU)):
    optimizerStr = "".join(optimizer.get_config()['name'].split("_")[:3])
  else:
    optimizerStr = str(optimizer)

  if (isinstance(regularizer, L1L2)):
    if (regularizer.get_config()['l1'] and regularizer.get_config()['l2']):
      egularizerStr = "l1l2"
    elif (regularizer.get_config()['l1']):
      regularizerStr = "l1"
    elif (regularizer.get_config()['l2']):
      regularizerStr = "l2"
    else:
      regularizerStr = "nan"
  else:
    regularizerStr = str(regularizer)

  temp1 = f"{str(owner)}_{str(modelName)}_{str(inputShape)}_{str(noOfClasses)}"
  temp1 += f"_{optimizerStr}_{str(batchSize)}_{str(dropout)}_{str(learnRatio)}"
  temp1 += f"_{str(hiddenActivation)}_{str(weightInitializer)}_{str(regularizerStr)}"
  temp2 = temp1 + f"_{str(iteration)}"

  name1 = f"{temp1}.hdf5"
  name2 = f"{temp2}.hdf5"
  figName = f"{temp2}.jpg"
  csvName = f"{temp1}.csv"

  checkpointFilepath1 = os.path.join(currentState["HDF5_RESULTS_DIR"], name1)
  checkpointFilepath2 = os.path.join(currentState["HDF5_RESULTS_DIR"], name2)
  figPath = os.path.join(currentState["FIGURES_RESULTS_DIR"], figName)
  csvPath = os.path.join(currentState["HISTORY_RESULTS_DIR"], csvName)

  modelCheckpointCallback1 = callbacks.ModelCheckpoint(
    filepath=checkpointFilepath1,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=0)
  csvLoggerCallback = callbacks.CSVLogger(csvPath, append=True)
  earlyStoppingCallback = callbacks.EarlyStopping(monitor='loss', patience=3)
  callbacksList = [
    modelCheckpointCallback1, csvLoggerCallback, earlyStoppingCallback
  ]

  toLog = f"New {temp1}."
  if (os.path.exists(checkpointFilepath1)):
    toLog = f"Updating using {temp1}."
    objs = {'f1_m': f1_m, 'precision_m': precision_m, 'recall_m': recall_m}
    model = load_model(checkpointFilepath1, custom_objects=objs)
    ev = model.evaluate(Xvalidation, Yvalidation, verbose=0)
    modelCheckpointCallback1.best = ev[1]
    message = f"Start the Model Checkpoint with a Best Value of {ev[1]}."
  else:
    ev = modelCheckpointCallback1.best
    message = f"Start the Model Checkpoint with a Best Value of {ev}."
    if (currentState["USE_TF"]):
      model = BuildCompileTransferLearningCNN(
        modelName,
        inputShape,
        noOfClasses,
        hiddenActivation,
        currentState["OUTPUT_ACTIVATION"],
        weightInitializer,
        currentState["PRETRAINED_WEIGHTS"],
        optimizer,
        dropout,
        learnRatio,
        regularizer,
        debug=currentState["DEBUG"])
    else:
      model = BuildCompileNativeCNN(modelName,
                      inputShape,
                      noOfClasses,
                      hiddenActivation,
                      currentState["OUTPUT_ACTIVATION"],
                      weightInitializer,
                      currentState["PRETRAINED_WEIGHTS"],
                      optimizer,
                      dropout,
                      learnRatio,
                      regularizer,
                      debug=currentState["DEBUG"])

  history = model.fit(trainIter,
            epochs=currentState["EPOCHS"],
            shuffle=True,
            validation_data=(Xvalidation, Yvalidation),
            callbacks=callbacksList,
            verbose=0)

  results = model.evaluate(Xtrain, Ytrain, verbose=0)
  first = copy.copy(results[:6])
  fitnessValueTrain = 0.1 * (1.0 / results[0]) + 0.5 * (results[1] * 100.0) + \
    0.1 * (results[2] * 100.0) + 0.1 * (results[3] * 100.0) + \
    0.1 * (results[4] * 100.0) + 0.1 * (results[5] * 100.0)
  results = model.evaluate(Xvalidation, Yvalidation, verbose=0)
  second = copy.copy(results[:6])
  fitnessValueVal = 0.1 * (1.0 / results[0]) + 0.5 * (results[1] * 100.0) + \
    0.1 * (results[2] * 100.0) + 0.1 * (results[3] * 100.0) + \
    0.1 * (results[4] * 100.0) + 0.1 * (results[5] * 100.0)
  results = model.evaluate(Xtest, Ytest, verbose=0)
  third = copy.copy(results[:6])
  fitnessValueTest = 0.1 * (1.0 / results[0]) + 0.5 * (results[1] * 100.0) + \
    0.1 * (results[2] * 100.0) + 0.1 * (results[3] * 100.0) + \
    0.1 * (results[4] * 100.0) + 0.1 * (results[5] * 100.0)
  totalFitnessValue = (fitnessValueTrain + fitnessValueVal +
             fitnessValueTest) / 3.0
  fitnessValue = np.round(totalFitnessValue, 5)

  record = [0, 0, 0, 0, 0, 0, 0]
  for q in range(6):
    record[q] = float((first[q] + second[q] + third[q]) / 3.0)
    if (q in [0, 2, 5]):
      if (record[q] >= 100): record[q] = round(record[q], 1)
      elif (record[q] >= 10): record[q] = round(record[q], 2)
      elif (record[q] >= 1): record[q] = round(record[q], 3)
      else: record[q] = round(record[q], 4)
    else:
      record[q] = round(record[q] * 100.0, 2)
  if (fitnessValue >= 100): record[-1] = 100
  elif (fitnessValue >= 10): record[-1] = round(fitnessValue, 2)
  elif (fitnessValue >= 1): record[-1] = round(fitnessValue, 3)
  else: record[-1] = round(fitnessValue, 4)

  del model, history, aug
  gc.collect()

  return fitnessValue, copy.copy(record)

In [ ]:
class PopulationBasedOptimizer(object):
  def __init__(self,
         fitnessFunction,
         currentState,
         lowerBound=0.0,
         upperBound=1.0):
    self.fitnessFunction = fitnessFunction
    self.currentState = currentState
    self.lowerBound = lowerBound
    self.upperBound = upperBound

    self.population = []
    self.populationWithScores = []
    self.currentIteration = 0

  def GetPopulation(self):
    return self.population.copy()

  def GetPopulationWithScores(self):
    return self.populationWithScores

  def InitPopulation(self):
    if (isinstance(self.currentState["DIM"], int)):
      R = self.currentState["DIM"]
    else:
      R = len(self.currentState["DIM"])
    if (not isinstance(self.lowerBound, list)):
      lb = [self.lowerBound for _ in range(R)]
    if (not isinstance(self.upperBound, list)):
      ub = [self.upperBound for _ in range(R)]
    lb = np.asarray(lb)
    ub = np.asarray(ub)

    size = (self.currentState["POPULATION_SIZE"], R)
    uniformRandom = np.random.uniform(0, 1, size)
    clippedUniformRandom = [x * (ub - lb) + lb for x in uniformRandom]
    self.population = np.round(np.asarray(clippedUniformRandom), 3)

  def CalculateFitnessScores(self):
    populationWithFitnessValues = []
    population = self.GetPopulation()
    for j, solution in enumerate(population):
      fitnessScore, record = self.fitnessFunction(
        self.currentIteration, solution, self.currentState)
      populationWithFitnessValues.append(
        [solution, fitnessScore, record])
    populationWithFitnessValues = sorted(populationWithFitnessValues,
                       key=lambda s: s[1],
                       reverse=True)
    self.populationWithScores = populationWithFitnessValues

  def Run(self):
    if (self.currentState["SKIP"] == 0
        and self.currentState["POPULATION"] is None):
      self.InitPopulation()
    else:
      self.population = self.currentState["POPULATION"]

    for i in range(self.currentState["SKIP"], self.currentState["ITERS"]):
      self.currentIteration = i + 1
      self.CalculateFitnessScores()
      self.LogHistory()
      self.UpdatePopulation()
      self.UpdateConfigurations()

  def LogHistory(self):
    for j, solution in enumerate(self.GetPopulationWithScores()):
      mappedSolution = MapSolutionForCNN(self.currentState, solution[0])

    bestSolution = self.GetPopulationWithScores()[0]
    mappedBestSolution = MapSolutionAsString(self.currentState,
                         bestSolution[0])
    mappedBestSolution.extend(bestSolution[2])

  def UpdateConfigurations(self):
    self.currentState["SKIP"] = self.currentIteration
    self.currentState["POPULATION"] = self.GetPopulation().copy()
    self.StoreWorkStateNow()

  def StoreWorkStateNow(self):
    StoreWorkState(self.currentState, self.currentState["STATE_PICKLE"])

  @abstractmethod
  def UpdatePopulation(self):
    raise NotImplementedError(
      "UpdatePopulation function must be implemented.")

In [ ]:
class GeneticAlgorithmOptimizer(PopulationBasedOptimizer):
  def UpdatePopulation(self):
    scores = self.GetPopulationWithScores()
    mappedSolutions = []
    for j, temp in enumerate(scores):
      mappedSolution = MapSolutionForCNN(self.currentState, temp[0])
      mappedSolutions.append(mappedSolution)

    Lc = len(scores)
    individuals = int(self.currentState["POPULATION_SIZE"] / 2.0)
    if (self.currentState["POPULATION_SIZE"] % 2 != 0): individuals += 1
    newPopulation = scores[:individuals].copy()
    newPopulation = [e[0].copy() for e in newPopulation]

    restCount = Lc - len(newPopulation)
    permutations = list(
      itertools.permutations(range(len(newPopulation)), 2)) * 3
    random.shuffle(permutations)
    if (isinstance(self.currentState["DIM"], int)):
      dim = self.currentState["DIM"]
    else:
      dim = len(self.currentState["DIM"])
    halfDim = int(dim / 2)
    cnt = 0

    while cnt < restCount:
      if (len(permutations) <= 0):
        permutations = list(
          itertools.permutations(range(len(newPopulation)), 2)) * 3
        random.shuffle(permutations)
      i, j = permutations.pop()

      newSolution = []
      newSolution.extend(newPopulation[i][:halfDim].copy())
      newSolution.extend(newPopulation[j][halfDim:].copy())

      if (np.random.random() >= 0.5):
        randomIndex = np.random.randint(0, dim)
        uniformRandom = np.random.uniform(0, 1)
        delta = self.upperBound - self.lowerBound
        clippedUniformRandom = uniformRandom * delta + self.lowerBound
        newSolution[randomIndex] = clippedUniformRandom

      newSolution = np.round(newSolution, 3)

      mappedSolution = MapSolutionForCNN(self.currentState, newSolution)
      if (mappedSolution not in mappedSolutions):
        newPopulation.append(newSolution)
        cnt += 1
      else:
        continue
    random.shuffle(newPopulation)
    self.population = np.round(np.asarray(newPopulation), 3)

In [ ]:
OWNER = "H.M.B"     
PBO   = "GA"        
USE_TF              = True   
MODEL               = "VGG16"      
DATASET_PICKLE_FILE = "PICKLE_NAME_HERE"
PICKELS_BASE_DIR    = "Pickels" 
RESULTS_BASE_DIR    = "Results" 
IMAGE_SHAPE         = (64, 64, 3)        
CATEGORIES          = ("Normal", "Pneumonia-Viral", "Pneumonia-Bacterial", "COVID-19")
ITERS               = 15      
EPOCHS              = 32      
POPULATION_SIZE     = 10     
DIM                 = 4
SKIP                = 0 
POPULATION          = None    
IS_NEW              = False   
TRAIN_RATIO         = 0.90    
DROPOUT_RANGE       = [0, 6]
OUTPUT_ACTIVATION   = "softmax"
PRETRAINED_WEIGHTS  = "imagenet"
BATCH_SIZES         = [32, 64, 32, 64, 32, 64, 32, 64, 32, 64, 32, 64]
LEARN_RATIOS        = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 100]
REGULARIZERS        = ["l1_1e-2", "l1_1e-3", "l1_1e-4", "l1_1e-5", "l2_1e-2", "l2_1e-3", "l2_1e-4", "l2_1e-5"]
OPTIMIZERS          = ["adam", "nadam", "adagrad", "adadelta", "adamax", "rmsprop", "sgd", "ftrl"]
HIDDEN_ACTIVATIONS  = ["relu", "tanh", "elu", "selu", "exponential", "prelu", "leakyrelu"]
WEIGHT_INITIALIZERS = [
  "glorot_normal", "glorot_uniform", "lecun_normal",
  "lecun_uniform", "he_normal", "he_uniform",
  "random_normal", "random_uniform", "truncated_normal"
]
AUGMENTATION_CONFIG = {
  "rotation_range": 15,
  "width_shift_range": 0.15,
  "height_shift_range": 0.15,
  "shear_range": 0.20,
  "zoom_range": 0.20,
  "horizontal_flip": True,
  "vertical_flip": False,
  "fill_mode": 'nearest'
}

In [ ]:
RESULTS_DIR = os.path.join(BASE_DIR, RESULTS_BASE_DIR)
DATASET_PICKLE = os.path.join(BASE_DIR, PICKELS_BASE_DIR, DATASET_PICKLE_FILE)
OWNER_DIR = os.path.join(RESULTS_DIR, OWNER)
if (not os.path.exists(OWNER_DIR)): os.mkdir(OWNER_DIR)
HDF5_RESULTS_DIR = os.path.join(OWNER_DIR, f"Models-{OWNER}")
FIGURES_RESULTS_DIR = os.path.join(OWNER_DIR, f"Figures-{OWNER}")
HISTORY_RESULTS_DIR = os.path.join(OWNER_DIR, f"History-{OWNER}")
STATE_PICKLE = os.path.join(OWNER_DIR, f"State-{OWNER}.p")
LOGGER_FILE = os.path.join(OWNER_DIR, f"Logger-{OWNER}.txt")
BEST_FILE = os.path.join(RESULTS_DIR, f"Best-{OWNER}.csv")
SPLIT_DATASET_PICKLE = os.path.join(BASE_DIR, PICKELS_BASE_DIR,
                  f"Split-{DATASET_PICKLE_FILE}")
if (not os.path.exists(HDF5_RESULTS_DIR)): os.mkdir(HDF5_RESULTS_DIR)
if (not os.path.exists(FIGURES_RESULTS_DIR)): os.mkdir(FIGURES_RESULTS_DIR)
if (not os.path.exists(HISTORY_RESULTS_DIR)): os.mkdir(HISTORY_RESULTS_DIR)

In [ ]:
currentState = HandleCurrentState(IS_NEW, STATE_PICKLE, LOGGER_FILE)
Q = LoadCreateSplitFile(currentState["DATASET_PICKLE"],
            currentState["SPLIT_DATASET_PICKLE"],
            len(currentState["CATEGORIES"]),
            currentState["TRAIN_RATIO"], currentState["DEBUG"])
[Xtrain, Xvalidation, Xtest, Ytrain, Yvalidation, Ytest] = Q

In [ ]:
gao = GeneticAlgorithmOptimizer(FitnessFunction, currentState, lowerBound=0.0, upperBound=1.0)
gao.Run()